In [1]:
import torch
import torch.onnx
import torch.nn as nn
import torch.optim as optim
import onnx  ## -> installled new
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# The path to the main directory 
main_dir = r'D:\Guvi\FinalPlantproj\Dataset\New Plant Diseases Dataset(Augmented)\train'

# List all classes in the main directory
classes = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

# Print the names of the classes
print("Classes:")
for class_name in classes:
    print(class_name)

Classes:
Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___healthy
Cherry_(including_sour)___Powdery_mildew
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___healthy
Corn_(maize)___Northern_Leaf_Blight
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___healthy
Strawberry___Leaf_scorch
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___healthy
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_mosaic_virus
Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [3]:
##   B4 running this remove working folder in dataset
# List the classes to work on
subdirs_to_copy = classes
# Create a new directory in the Kaggle working directory
subset_dir = 'D:\Guvi\FinalPlantproj\Dataset\working'
os.makedirs(subset_dir, exist_ok=True)

# Copy the selected classes to the new directory
for subdir_name in subdirs_to_copy:
    src_dir_path = os.path.join(main_dir, subdir_name)
    dest_dir_path = os.path.join(subset_dir, subdir_name)
    shutil.copytree(src_dir_path, dest_dir_path)

print(f"Subdirectories {subdirs_to_copy} copied to {subset_dir}")

Subdirectories ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotte

In [4]:
# Initialize a counter for the number of images
image_count = 0

# Iterate over all classes and count the images
for subdir, _, files in os.walk(subset_dir):
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
    image_count += len(image_files)

print(f"Total number of images: {image_count}")

Total number of images: 70295


### data preprocessing

In [5]:
dataset_path = 'D:\Guvi\FinalPlantproj\Dataset\working'

In [6]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),              # resize all images to 150x150 pixels
    transforms.ToTensor(),                      # convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [8]:
dataset.classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

### train test split 

In [9]:
train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=0.3, random_state=42)

train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

In [10]:
trainloader = DataLoader(train_dataset, batch_size=15, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=15, shuffle=True)

### define cnn model

In [11]:
class CNN_Classification(nn.Module):
    def __init__(self):
        super(CNN_Classification, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)       # first convolutional layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)            # first pooling layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)      # second convolutional layer
        self.fc1 = nn.Linear(64 * 37 * 37, 128)                                 # 37 x 37 is the size after pooling
        self.fc2 = nn.Linear(128, len(dataset.classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)                                                 # flatten all dimensions
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
CNN_Model = CNN_Classification()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_Model.parameters(), lr=0.001)          # Defines the optimizer, Adam with a learning rate of 0.001

### train the model

In [14]:
num_epochs = 10
for epoch in range(num_epochs):

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = CNN_Model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


print('Finished Training')

Epoch 1/10, Loss: 0.3677082061767578
Epoch 2/10, Loss: 0.4519745111465454
Epoch 3/10, Loss: 0.03265060484409332
Epoch 4/10, Loss: 0.024218380451202393
Epoch 5/10, Loss: 0.0038494367618113756
Epoch 6/10, Loss: 1.0418986082077026
Epoch 7/10, Loss: 0.004178566392511129
Epoch 8/10, Loss: 8.437884389422834e-05
Epoch 9/10, Loss: 9.418846457265317e-05
Epoch 10/10, Loss: 0.0017746221274137497
Finished Training


### evaluation

In [15]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation mode
CNN_Model.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = CNN_Model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')

Accuracy: 86.98 %
Precision: 87.5391%
Recall: 86.9837%
F1-score: 86.9378%


### Prediction

In [16]:
# Load a test image and preprocess it
img = Image.open('D:\Guvi\FinalPlantproj\ANUKSHMITHA0610\TomatoHealty.JPG')
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {dataset.classes[predicted.item()]}')

Predicted class: Tomato___healthy


In [17]:
# Load a test image and preprocess it
img = Image.open(r'D:\Guvi\FinalPlantproj\Dataset\New Plant Diseases Dataset(Augmented)\valid\Grape___Esca_(Black_Measles)\02af0429-46c1-444b-bf62-a4d0198141e8___FAM_B.Msls 1062.JPG')
img = transform(img).unsqueeze(0)  # add batch dimension

# Pass the image through the model
CNN_Model.eval()
output = CNN_Model(img)
_, predicted = torch.max(output, 1)
print(f'Predicted class: {dataset.classes[predicted.item()]}')

Predicted class: Grape___Esca_(Black_Measles)


In [18]:
torch.save(CNN_Model.state_dict(), 'cnn_model.pth')  

In [19]:
CNN_Model.load_state_dict(torch.load('cnn_model.pth'))

<All keys matched successfully>

### Pretrained models

#### Vgg16

In [21]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation mode
vgg16.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []
  
# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = vgg16(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.4f}%')
print(f'Recall: {recall * 100:.4f}%')
print(f'F1-score: {f1 * 100:.4f}%')

Accuracy: 84.89 %
Precision: 86.2847%
Recall: 84.8879%
F1-score: 84.4643%


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Transfer model to the GPU
vgg16 = vgg16.to(device)

# Freeze all the layers (optional, if you don't want to train the convolutional layers)
for param in vgg16.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
vgg16.classifier[6] = nn.Linear(4096, len(dataset.classes))  # 4096 is the input to the last layer

# Transfer classifier changes to the GPU
vgg16.classifier = vgg16.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move criterion to GPU
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = vgg16(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training VGG16')

Using device: cpu


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\user/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [00:50<00:00, 11.0MB/s]


Epoch 1/5, Loss: 1.042962670326233
Epoch 2/5, Loss: 0.7125754356384277
Epoch 3/5, Loss: 1.4591518640518188
Epoch 4/5, Loss: 2.754568099975586
Epoch 5/5, Loss: 0.8263623118400574
Finished Training VGG16


### DenseNet

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained DenseNet121 model
#densenet = models.densenet121(pretrained=True)
densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)

# Transfer model to the GPU
densenet = densenet.to(device)

# Freeze all the layers if you don't want to train the convolutional layers (optional)
for param in densenet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
num_ftrs = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_ftrs, len(dataset.classes))

# Transfer classifier changes to the GPU
densenet.classifier = densenet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(densenet.parameters(), lr=0.02)

# Train the model
num_epochs = 4
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = densenet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training DenseNet')

Using device: cpu


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\user/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|█████████████████████████████████████████████████████████████████████████████| 30.8M/30.8M [00:02<00:00, 11.8MB/s]


Epoch 1/4, Loss: 6.621461391448975
Epoch 2/4, Loss: 6.416478633880615
Epoch 3/4, Loss: 12.288323402404785
Epoch 4/4, Loss: 7.947284075271455e-08
Finished Training DenseNet


In [23]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
# densenet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = densenet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')


Accuracy: 89.05 %
Precision: 90.49%
Recall: 89.05%
F1-score: 89.07%


### AlexNet

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)
#alexnet = models.alexnet(weights=models.alexnet_Weights.DEFAULT)


# Transfer model to the GPU
alexnet = alexnet.to(device)

# Freeze layers if desired
for param in alexnet.parameters():
    param.requires_grad = False

# Modify the classifier to fit the number of classes in your dataset
alexnet.classifier[6] = nn.Linear(4096, len(dataset.classes))

# Transfer classifier changes to the GPU
alexnet.classifier = alexnet.classifier.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move loss function to GPU
optimizer = optim.Adam(alexnet.parameters(), lr=0.02)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in trainloader:
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print('Finished Training AlexNet')

Using device: cpu
Epoch 1/5, Loss: 0.0
Epoch 2/5, Loss: 0.0
Epoch 3/5, Loss: 23.71910285949707
Epoch 4/5, Loss: 25.55242347717285
Epoch 5/5, Loss: 71.16920471191406
Finished Training AlexNet


In [28]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Switch to evaluation model
alexnet.eval()

# Initialize variables for storing predictions and true labels
all_preds = []
all_labels = []

# Correct and total for accuracy
correct = 0
total = 0

# No gradients needed during evaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
      
        # Forward pass to get predictions
        outputs = alexnet(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        
        # Update accuracy calculations
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        
        # Collect predictions and true labels for other metrics
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f} %')

# Calculate precision, recall, and F1-score using sklearn
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')

Accuracy: 88.07 %
Precision: 88.75%
Recall: 88.07%
F1-score: 88.00%


In [29]:
torch.save(vgg16.state_dict(), 'vgg16.pth')  
torch.save(alexnet.state_dict(), 'alexnet_model.pth') 
torch.save(densenet.state_dict(), 'densenet.pth')